In [1]:
import os
os.chdir('/content/drive/MyDrive/000GithubRepos/bigcontest2024')

In [2]:
pip install python-dotenv

In [3]:
pip install langchain_community

In [4]:
pip install sentence-transformers

In [5]:
pip install neo4j

In [6]:
pip install neo4j_genai

In [7]:
pip install openai

In [8]:
pip install langchain_huggingface

In [9]:
pip install langchain_google_genai

In [10]:
pip install openai

In [11]:
from cypher_query.retrieval_query import retrievalQuery
from dotenv import load_dotenv
from config import CONFIG

from graphrag.get_embedding_model import get_embedding_model
from langchain_community.vectorstores.neo4j_vector import Neo4jVector

load_dotenv()

True

In [12]:
def get_neo4j_vector(index_name='queryVector'):
    neo4jvector = Neo4jVector.from_existing_index(
        embedding=get_embedding_model(),  # Using the custom embedding function
        url=CONFIG.neo4j_url,
        database='neo4j',
        username=CONFIG.neo4j_user,
        password=CONFIG.neo4j_password,
        index_name=index_name,
        text_node_property="textEmbedding",
        retrieval_query=retrievalQuery
    )
    return neo4jvector

store_retriever = get_neo4j_vector().as_retriever(search_kwargs={"k": 5})

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
from graphrag.retriever import retrieve_store_nodes

In [14]:
query = "연인"

In [15]:
response = retrieve_store_nodes(query)

In [16]:
response

[Document(metadata={'storeName': '나원회포차', 'reviewText': '음식 나오기까지 다소 시간이 걸리긴하지만 여자친구가 딱새우회를 사랑하는데 그 딱새우회와 매운탕이 진짜 기가 막히게 맛있었습니다 그리고 밑반찬도 맛있었고요 잘 먹고 가요~!!!', 'score': 0.6874282360076904, 'storeAddress': '제주 서귀포시 서귀동 254-6번지', 'storeType': '일식'}, page_content='음식 나오기까지 다소 시간이 걸리긴하지만 여자친구가 딱새우회를 사랑하는데 그 딱새우회와 매운탕이 진짜 기가 막히게 맛있었습니다 그리고 밑반찬도 맛있었고요 잘 먹고 가요~!!!'),
 Document(metadata={'storeName': '나폴리피자', 'reviewText': '사실 바다앞이고 풍경좋아서 갔는데 여자친구가 너무 맛있다고 좋아함. 피자가 2.5~3만원 대 파스타가 1.5~2만원대라서 나쁘지 않음.', 'score': 0.6625118255615234, 'storeAddress': '제주 서귀포시 안덕면 창천리 912번지', 'storeType': '피자'}, page_content='사실 바다앞이고 풍경좋아서 갔는데 여자친구가 너무 맛있다고 좋아함. 피자가 2.5~3만원 대 파스타가 1.5~2만원대라서 나쁘지 않음.'),
 Document(metadata={'storeName': '구럼비나무', 'reviewText': '맛있어여', 'score': 0.6305379867553711, 'storeAddress': '제주 서귀포시 법환동 218번지 1층', 'storeType': '단품요리 전문'}, page_content='맛있어여'),
 Document(metadata={'storeName': '교촌치킨탑동점', 'reviewText': '맛있는편', 'score': 0.6250295639038086, 'storeAddress': '제주 제주시 건입동 1441번지', 'storeType

In [17]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [18]:
from neo4j import GraphDatabase
uri = os.environ["NEO4J_URI"]
username = os.environ["NEO4J_USERNAME"]
password = os.environ["NEO4J_PASSWORD"]
driver = GraphDatabase.driver(uri, auth=(username, password))

In [19]:
from llm_response.get_llm_model import get_llm_model

llm = get_llm_model()

In [ ]:
"""
Node properties:
Person {name: STRING, born: INTEGER}
Movie {tagline: STRING, title: STRING, released: INTEGER}

Relationship properties:
ACTED_IN {roles: LIST}
REVIEWED {summary: STRING, rating: INTEGER}

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:PRODUCED]->(:Movie)
(:Person)-[:WROTE]->(:Movie)
(:Person)-[:FOLLOWS]->(:Person)
(:Person)-[:REVIEWED]->(:Movie)"""

In [20]:
import os
os.getcwd()

'/content/drive/MyDrive/000GithubRepos/bigcontest2024'

In [155]:
neo4j_schema = """
Node properties:
STORE {
  MCT_NM: STRING. 가맹점명. ex) "토평골식당",
  ADDR: STRING. 가맹점 주소. ex) "제주 서귀포시 토평동 1245-7번지",
  MCT_TYPE: STRING. 요식관련 30개업종. ex) '가정식', '단품요리 전문', '커피', '베이커리', '일식', '치킨', '중식', '분식', '햄버거', '양식', '맥주/요리주점', '아이스크림/빙수', '피자', '샌드위치/토스트', '차', '꼬치구이', '기타세계요리', '구내식당/푸드코트', '떡/한과', '도시락', '도너츠', '주스', '동남아/인도음식', '패밀리 레스토랑', '기사식당', '야식', '스테이크', '포장마차', '부페', '민속주점'
  OP_YMD: STRING. 가맹점개설일자. ex)"20050704"
}
MONTH {
  YM: STRING. 기준연월. ex) "202306",
  month: STRING. 월 이름. ex) "June"
}
Region {
  name: STRING. 지역명. ex) "대포동", "서홍동", "대정읍"
}
City {
  name: STRING. 도시명. ex) "서귀포시", "제주시"
}

Relationship properties:
USE {
  UE_CNT_GRP: 이용건수구간. 월별 이용건수를 6개 구간으로 집계 시, 해당 가맹점의 이용건수가 포함되는 분위수 구간. ex)'상위 10% 이하', '10~25%', '25~50%', '50~75%', '75~90%', '90% 초과(하위 10% 이하)'
  UE_AMT_GRP: 이용금액구간. 월별 이용금액을 6개 구간으로 집계 시, 해당 가맹점의 이용금액이 포함되는 분위수 구간. ex)'상위 10% 이하', '10~25%', '25~50%', '50~75%', '75~90%', '90% 초과(하위 10% 이하)'
  UE_AMT_PER_TRSN_GRP: 건당 평균 이용금액 구간. 월별 건당 평균 이용금액을 6개 구간으로 집계 시, 해당 가맹점의 건당 평균 이용금액이 포함되는 분위수 구간. ex)'상위 10% 이하', '10~25%', '25~50%', '50~75%', '75~90%', '90% 초과(하위 10% 이하)'
  MON_UE_CNT_RAT: 월요일 이용건수 비중. FLOAT. ex) 0.1262
  TUE_UE_CNT_RAT: 화요일 이용건수 비중. FLOAT. ex) 0.1262
  WED_UE_CNT_RAT: 수요일 이용건수 비중. FLOAT. ex) 0.1941
  THU_UE_CNT_RAT: 목요일 이용건수 비중. FLOAT. ex) 0.2233
  FRI_UE_CNT_RAT: 금요일 이용건수 비중. FLOAT. ex) 0.1553
  SAT_UE_CNT_RAT: 토요일 이용건수 비중. FLOAT. ex) 0.1747
  SUN_UE_CNT_RAT: 일요일 이용건수 비중. FLOAT. ex) 0.1818

  # 시간대별
  HR_5_11_UE_CNT_RAT: 5시~11시 이용건수 비중. FLOAT. ex) 0.1650
  HR_12_13_UE_CNT_RAT: 12시~13시 이용건수 비중. FLOAT. ex) 0.5242
  HR_14_17_UE_CNT_RAT: 14시~17시 이용건수 비중. FLOAT. ex) 0.3107
  HR_18_22_UE_CNT_RAT: 18시~22시 이용건수 비중. FLOAT. ex) 0.0511
  HR_23_4_UE_CNT_RAT: 23시~4시 이용건수 비중. FLOAT. ex) 0.07313

  # 현지인
  LOCAL_UE_CNT_RAT: 현지인 이용건수 비중. FLOAT. ex) 0.5843

  # 성별
  RC_M12_MAL_CUS_CNT_RAT: 남성 이용건수 비중. FLOAT. ex) 0.634
  RC_M12_FME_CUS_CNT_RAT: 여성 이용건수 비중. FLOAT. ex) 0.366

  # 연령대별
  RC_M12_AGE_UND_20_CUS_CNT_RAT: 20대 이하 이용건수 비중. FLOAT. ex) 0.066
  RC_M12_AGE_30_CUS_CNT_RAT: 30대 이용건수 비중. FLOAT. ex) 0.252
  RC_M12_AGE_40_CUS_CNT_RAT: 40대 이용건수 비중. FLOAT. ex) 0.398
  RC_M12_AGE_50_CUS_CNT_RAT: 50대 이용건수 비중. FLOAT. ex) 0.201
  RC_M12_AGE_OVR_60_CUS_CNT_RAT: 60대 이상 이용건수 비중. FLOAT. ex) 0.083
}
HAS_STORE {}
HAS_REGION {}

The relationships:
(:City)-[:HAS_REGION]->(:Region)
(:REGION)-[:HAS_STORE]->(:STORE)
(:STORE)-[:USE]->(:MONTH)
"""

In [157]:
examples = [
    """USER INPUT: '제주시 한림읍에 있는 카페 중 30대 이용 비중이 가장 높은 곳은?' QUERY: MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시'
  AND r.name = '한림읍'
  AND s.MCT_TYPE = "커피"
WITH s, avg(u.RC_M12_AGE_30_CUS_CNT_RAT) AS avg_age_30_ratio
RETURN s.MCT_NM, avg_age_30_ratio
ORDER BY avg_age_30_ratio DESC
LIMIT 1""",
    """USER INPUT: '제주시 노형동에 있는 단품요리 전문점 중 이용건수가 상위 10%에 속하고 현지인 이용 비중이 가장 높은 다섯 곳은?' QUERY: MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시'
  AND r.name = '노형동'
  AND s.MCT_TYPE = '단품요리 전문'
  AND u.UE_CNT_GRP = '상위 10% 이하'
WITH s, avg(u.LOCAL_UE_CNT_RAT) AS avg_local_ratio
RETURN s.MCT_NM, avg_local_ratio
ORDER BY avg_local_ratio DESC
LIMIT 5
"""
]

examples_combined = '\n'.join(examples) if examples else ''

In [158]:
t2c_prompt_template = """
Task: Generate a Cypher statement for querying a Neo4j graph database from a user input.

Schema:
{neo4j_schema}

Examples (optional):
{examples_combined}

Input:
{query}

Never use any properties or relationships not included in the schema.
Never include triple backticks ```.

Cypher query:"""

In [167]:
query = '제주시 노형동에 있는 단품요리 전문점 중 이용건수가 상위 10%에 속하고 현지인 이용 비중이 가장 높은 곳은?'
query = "제주시 한림읍에 있는 카페 중 30대 이용 비중이 가장 높은 곳은?"
query = "삼도일동에 있는 빵집 중 오후 7시 이용 비중이 가장 높은 곳은?"

In [168]:
print(f"query : {query}")

print(f"Text2Cypher".ljust(100, '-'))
response = llm.invoke(t2c_prompt_template.format(neo4j_schema=neo4j_schema, examples_combined=examples_combined, query=query))
cypher = response.content.replace('```', '').replace('cypher', '').strip()
print(cypher)

query : 삼도일동에 있는 빵집 중 오후 7시 이용 비중이 가장 높은 곳은?
Text2Cypher-----------------------------------------------------------------------------------------
MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시'
  AND r.name = '삼도일동'
  AND s.MCT_TYPE = '베이커리'
WITH s, avg(u.HR_19_22_UE_CNT_RAT) AS avg_hr_19_22_ratio
RETURN s.MCT_NM, avg_hr_19_22_ratio
ORDER BY avg_hr_19_22_ratio DESC
LIMIT 1


In [169]:
print(f"Cypher result".ljust(100, '-'))
records, summary, keys = driver.execute_query(cypher, )

Cypher result---------------------------------------------------------------------------------------


In [170]:
records

[<Record s.MCT_NM='공설' avg_hr_19_22_ratio=None>]

In [171]:
summary.notifications

[{'title': 'The provided property key is not in the database',
  'code': 'Neo.ClientNotification.Statement.UnknownPropertyKeyWarning',
  'description': "One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: HR_19_22_UE_CNT_RAT)",
  'severity': 'WARNING',
  'category': 'UNRECOGNIZED',
  'position': {'offset': 167, 'line': 5, 'column': 15}},
 {'title': 'The query contains an aggregation function that skips null values.',
  'code': 'Neo.ClientNotification.Statement.AggregationSkippedNull',
  'description': 'null value eliminated in set function.',
  'severity': 'WARNING',
  'category': 'UNRECOGNIZED'}]

In [154]:
keys

['s.MCT_NM', 'avg_hr_19_22_ratio']

In [ ]:
rout_intt_prompt_template = f"""Classify the query as either a recommendation question (recomm) or a search question (search).

query : {query}

example)
query1 : "제주시 한림읍에 있는 카페 중 30대 이용 비중이 가장 높은 곳은?"
answer1 : "{
    'query_type' : 'search',
    ''
}"

query2 : "60대 부부가 가기 좋은 흑돼지 식당 추천해줘"
"{
    'query_type' : 'recomm',
    ''
}"

"""

In [142]:

record_str_lst = []
for record in records:
    record_str_lst.append(", ".join([f"{k} : {v}" for k, v in record.items()]))
search_result = '\n'.join(record_str_lst)
print(search_result)

formatting_prompt = f"""Answer to the user's question refer to cypher query and the search result.

user's question : {query}

cypher query : {cypher}

search result : {search_result}

- Explain how the result was derived with reasons.
- Not technically, but kindly to general people who questioned."""
search_question_response = llm.invoke(formatting_prompt)

print(f"Final answer".ljust(100, '-'))
print(search_question_response.content)

query : 삼도일동에 있는 빵집 중 오후 7시 이용 비중이 가장 높은 곳은?
Text2Cypher-----------------------------------------------------------------------------------------


MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시'
  AND r.name = '삼도일동'
  AND s.MCT_TYPE = '베이커리'
WITH s, avg(u.HR_19_22_UE_CNT_RAT) AS avg_hr_19_22_ratio
RETURN s.MCT_NM, avg_hr_19_22_ratio
ORDER BY avg_hr_19_22_ratio DESC
LIMIT 1
Cypher result---------------------------------------------------------------------------------------
s.MCT_NM : 공설, avg_hr_19_22_ratio : None
Final answer----------------------------------------------------------------------------------------
## 삼도일동에서 오후 7시 이용 비중이 가장 높은 빵집은 "공설"입니다!

저희 데이터베이스를 분석한 결과, 삼도일동 지역의 빵집 중 "공설"이라는 곳이 오후 7시 이용 비중이 가장 높은 것으로 나타났습니다. 

데이터 분석 결과 "공설"의 오후 7시 이용 비중은 다른 빵집들보다 높았지만, 정확한 수치는 아직 확인되지 않았습니다. 

앞으로 더 많은 데이터를 수집하고 분석하여 더 정확한 정보를 제공해드리도록 노력하겠습니다. 



In [ ]:
import pandas as pd

# 엑셀 파일을 불러오는 코드
file_path = 'data/JEJU_MCT_DATA_v2.csv'  # 파일 경로 설정
df = pd.read_csv(file_path, encoding='cp949')

In [ ]:
sorted(df['UE_CNT_GRP'].unique())

In [ ]:
sorted(df['UE_AMT_PER_TRSN_GRP'].unique())

In [ ]:
df['MCT_TYPE'].unique()

In [73]:
samdoildong = df[df['ADDR'].map(lambda x: '삼도일동' in x)]

In [77]:
bakery = samdoildong[samdoildong['MCT_TYPE'] == '베이커리']

In [78]:
bakery

,YM,MCT_NM,OP_YMD,MCT_TYPE,ADDR,UE_CNT_GRP,UE_AMT_GRP,UE_AMT_PER_TRSN_GRP,MON_UE_CNT_RAT,TUE_UE_CNT_RAT,...,HR_18_22_UE_CNT_RAT,HR_23_4_UE_CNT_RAT,LOCAL_UE_CNT_RAT,RC_M12_MAL_CUS_CNT_RAT,RC_M12_FME_CUS_CNT_RAT,RC_M12_AGE_UND_20_CUS_CNT_RAT,RC_M12_AGE_30_CUS_CNT_RAT,RC_M12_AGE_40_CUS_CNT_RAT,RC_M12_AGE_50_CUS_CNT_RAT,RC_M12_AGE_OVR_60_CUS_CNT_RAT
1221,202301,뚜레쥬르서사라점,20150406,베이커리,제주 제주시 삼도일동 535-14번지,1_상위 10% 이하,3_25~50%,5_75~90%,0.140221,0.110701,...,0.394834,0.003690,0.738575,0.454,0.546,0.134,0.256,0.304,0.203,0.102
1490,202301,파리바게트 제주서사라점,20210408,베이커리,제주 제주시 삼도일동 519-15번지 세종빌딩 1층,1_상위 10% 이하,3_25~50%,6_90% 초과(하위 10% 이하),0.193309,0.081784,...,0.386617,0.000000,0.760171,0.419,0.581,0.123,0.199,0.292,0.237,0.148
4764,202301,행복밀,20201106,베이커리,제주 제주시 삼도일동 553-4번지 1층,1_상위 10% 이하,2_10~25%,5_75~90%,0.151300,0.115839,...,0.205674,0.004728,0.571993,0.371,0.629,0.201,0.293,0.229,0.183,0.093
7642,202302,행복밀,20201106,베이커리,제주 제주시 삼도일동 553-4번지 1층,1_상위 10% 이하,2_10~25%,5_75~90%,0.157044,0.159353,...,0.237875,0.000000,0.568875,0.370,0.630,0.200,0.293,0.225,0.186,0.095
7788,202302,파리바게트 제주서사라점,20210408,베이커리,제주 제주시 삼도일동 519-15번지 세종빌딩 1층,2_10~25%,3_25~50%,6_90% 초과(하위 10% 이하),0.186508,0.150794,...,0.345238,0.000000,0.757675,0.419,0.581,0.120,0.192,0.288,0.250,0.150
8496,202302,뚜레쥬르서사라점,20150406,베이커리,제주 제주시 삼도일동 535-14번지,2_10~25%,3_25~50%,6_90% 초과(하위 10% 이하),0.128099,0.148760,...,0.376033,0.000000,0.739829,0.455,0.545,0.125,0.258,0.306,0.209,0.102
11603,202303,뚜레쥬르서사라점,20150406,베이커리,제주 제주시 삼도일동 535-14번지,2_10~25%,3_25~50%,5_75~90%,0.097166,0.178138,...,0.368421,0.000000,0.748107,0.452,0.548,0.121,0.255,0.312,0.210,0.102
13502,202303,파리바게트 제주서사라점,20210408,베이커리,제주 제주시 삼도일동 519-15번지 세종빌딩 1층,1_상위 10% 이하,3_25~50%,6_90% 초과(하위 10% 이하),0.142373,0.179661,...,0.291525,0.000000,0.766111,0.421,0.579,0.114,0.194,0.290,0.253,0.149
16945,202303,행복밀,20201106,베이커리,제주 제주시 삼도일동 553-4번지 1층,1_상위 10% 이하,2_10~25%,5_75~90%,0.120388,0.147573,...,0.213592,0.001942,0.560217,0.370,0.630,0.204,0.289,0.226,0.186,0.095
17319,202304,파리바게트 제주서사라점,20210408,베이커리,제주 제주시 삼도일동 519-15번지 세종빌딩 1층,1_상위 10% 이하,3_25~50%,6_90% 초과(하위 10% 이하),0.149153,0.122034,...,0.325424,0.000000,0.762277,0.425,0.575,0.111,0.193,0.287,0.259,0.150


In [84]:
jan = df[df['YM'] == 202301]

In [87]:
len(jan)

5808

In [44]:
from neo4j_genai.retrievers import Text2CypherRetriever

retriever = Text2CypherRetriever(
    driver=driver,
    llm=llm,  # type: ignore
    # neo4j_schema=neo4j_schema,
    # examples=examples,
    custom_prompt=custom_prompt
)

In [45]:
search_result = retriever.search(query_text=query)

Text2CypherRetrievalError: Failed to get search result: Invalid input '```cypher\nMATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)\nWHERE c.name = '제주시'\n  AND r.name = '삼도일동'\n  AND s.MCT_TYPE = '베이커리'\nWITH s, avg(u.WED_UE_CNT_RAT) AS avg_wed_ratio\nRETURN s.MCT_NM, avg_wed_ratio\nORDER BY avg_wed_ratio DESC\nLIMIT 1\n```': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"```cypher"
 ^

# mct 데이터 확인

In [39]:
hanlim = df[df['ADDR'].map(lambda x: '한림' in x)]

NameError: name 'df' is not defined

In [ ]:
coffee = hanlim[hanlim['MCT_TYPE'] == '커피']

In [ ]:
coffee.sort_values(by='RC_M12_AGE_30_CUS_CNT_RAT', ascending=False)[['MCT_NM', 'MCT_TYPE', 'YM', 'RC_M12_AGE_30_CUS_CNT_RAT']].head(50)

,MCT_NM,MCT_TYPE,YM,RC_M12_AGE_30_CUS_CNT_RAT
25203,나이스데이밀,커피,202305,0.439
38802,나이스데이밀,커피,202307,0.431
30978,나이스데이밀,커피,202306,0.426
43510,나이스데이밀,커피,202308,0.424
48388,나이스데이밀,커피,202309,0.418
55825,나이스데이밀,커피,202310,0.414
21617,나이스데이밀,커피,202304,0.405
13779,나이스데이밀,커피,202303,0.400
24339,벌크커피한림점,커피,202305,0.381
18181,벌크커피한림점,커피,202304,0.381


In [ ]:
nohyeong = df[df['ADDR'].map(lambda x: '노형' in x)]
nohyeong

,YM,MCT_NM,OP_YMD,MCT_TYPE,ADDR,UE_CNT_GRP,UE_AMT_GRP,UE_AMT_PER_TRSN_GRP,MON_UE_CNT_RAT,TUE_UE_CNT_RAT,...,HR_18_22_UE_CNT_RAT,HR_23_4_UE_CNT_RAT,LOCAL_UE_CNT_RAT,RC_M12_MAL_CUS_CNT_RAT,RC_M12_FME_CUS_CNT_RAT,RC_M12_AGE_UND_20_CUS_CNT_RAT,RC_M12_AGE_30_CUS_CNT_RAT,RC_M12_AGE_40_CUS_CNT_RAT,RC_M12_AGE_50_CUS_CNT_RAT,RC_M12_AGE_OVR_60_CUS_CNT_RAT
4,202301,까망꼬숑,20220826,단품요리 전문,제주 제주시 노형동 3784-12번지 1층,5_75~90%,3_25~50%,2_10~25%,0.272727,0.121212,...,0.727273,0.090909,0.264463,0.513,0.487,0.412,0.311,0.176,0.076,0.025
17,202301,오리정,20000128,단품요리 전문,제주 제주시 노형동 1288-5번지,4_50~75%,4_50~75%,3_25~50%,0.138889,0.138889,...,0.611111,0.000000,0.779747,0.606,0.394,0.118,0.233,0.173,0.313,0.163
23,202301,참맛나,20161027,분식,제주 제주시 노형동 3813-5번지 1층,1_상위 10% 이하,1_상위 10% 이하,5_75~90%,0.152918,0.169014,...,0.006036,0.000000,0.711680,0.464,0.536,0.170,0.342,0.303,0.148,0.037
26,202301,전철우고향랭면해장국 신제주직영점,20220523,단품요리 전문,제주 제주시 노형동 2282번지 1층,4_50~75%,5_75~90%,4_50~75%,0.022222,0.222222,...,0.000000,0.000000,0.742354,0.586,0.414,0.036,0.137,0.281,0.292,0.254
35,202301,생돈구이촌,20110405,가정식,제주 제주시 노형동 1277-5번지,6_90% 초과(하위 10% 이하),5_75~90%,2_10~25%,0.200000,0.133333,...,0.866667,0.000000,0.485981,0.694,0.306,0.131,0.311,0.296,0.223,0.039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67838,202312,짬뽕지존제주점,20221026,중식,제주 제주시 노형동 3806-6번지 1층 2호,3_25~50%,4_50~75%,4_50~75%,0.118280,0.172043,...,0.290323,0.000000,0.644567,0.662,0.338,0.174,0.265,0.273,0.210,0.079
67845,202312,물만난고기,20230130,단품요리 전문,제주 제주시 노형동 1276-5번지 1층,5_75~90%,3_25~50%,2_10~25%,0.153846,0.115385,...,0.961538,0.000000,0.645051,0.754,0.246,0.064,0.113,0.202,0.312,0.309
67846,202312,테이크파이브,20230918,단품요리 전문,제주 제주시 노형동 156-8번지 1층,4_50~75%,3_25~50%,2_10~25%,0.051282,0.025641,...,0.897436,0.102564,0.450549,0.413,0.587,0.283,0.304,0.207,0.174,0.033
67861,202312,경일낙지,20020112,단품요리 전문,제주 제주시 노형동 1291-3번지,2_10~25%,2_10~25%,4_50~75%,0.179310,0.241379,...,0.000000,0.000000,0.825083,0.424,0.576,0.054,0.218,0.379,0.263,0.086


In [ ]:
nohyeong['MCT_TYPE'].value_counts()

,count
MCT_TYPE,
단품요리 전문,2222
가정식,1653
커피,407
치킨,219
일식,198
중식,167
분식,163
베이커리,155
맥주/요리주점,141


In [ ]:
single_cuisine = nohyeong[nohyeong['MCT_TYPE'] == '단품요리 전문']

In [ ]:
upper_10 = single_cuisine[single_cuisine['UE_CNT_GRP'] == '1_상위 10% 이하']

In [ ]:
upper_10.sort_values(by='LOCAL_UE_CNT_RAT', ascending=False)[['MCT_NM', 'MCT_TYPE', 'YM', 'LOCAL_UE_CNT_RAT', 'UE_CNT_GRP']].head(20)

,MCT_NM,MCT_TYPE,YM,LOCAL_UE_CNT_RAT,UE_CNT_GRP
52072,블루메베이글노형점,단품요리 전문,202310,0.829837,1_상위 10% 이하
65592,샤브마니아제주노형점,단품요리 전문,202312,0.826381,1_상위 10% 이하
60592,샤브마니아제주노형점,단품요리 전문,202311,0.824624,1_상위 10% 이하
11512,샤브마니아제주노형점,단품요리 전문,202302,0.823994,1_상위 10% 이하
58215,돈까스가있는풍경,단품요리 전문,202311,0.823697,1_상위 10% 이하
14958,샤브마니아제주노형점,단품요리 전문,202303,0.822757,1_상위 10% 이하
46404,샤브마니아제주노형점,단품요리 전문,202309,0.822129,1_상위 10% 이하
21445,샤브마니아제주노형점,단품요리 전문,202304,0.822127,1_상위 10% 이하
1737,샤브마니아제주노형점,단품요리 전문,202301,0.821859,1_상위 10% 이하
50405,24시 전주명가콩나물국밥 노형점,단품요리 전문,202309,0.811971,1_상위 10% 이하


# Text2Cypher 실행 테스트

In [33]:
import logging

# 로그 레벨 설정 (DEBUG 레벨 이상 출력)
logging.basicConfig(level=logging.DEBUG)

# 로그 포맷 설정 (선택 사항)
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.DEBUG
)


In [34]:


search_result = retriever.search(query_text=query)

Text2CypherRetrievalError: Failed to get search result: Invalid input 'Please': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"Please provide the following information:"
 ^

In [ ]:
print(search_result.metadata['cypher'])

MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시' AND r.name = '삼도일동' AND s.MCT_TYPE = '베이커리'
RETURN s.MCT_NM
ORDER BY s.MCT_NM
LIMIT 10


In [ ]:
print("""MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)\nWHERE c.name = '제주시' AND r.name = '삼도일동' AND s.MCT_TYPE = '베이커리'\nRETURN s.MCT_NM, u.WED_UE_CNT_RAT\nORDER BY u.WED_UE_CNT_RAT DESC\nLIMIT 1\n""")

MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시' AND r.name = '삼도일동' AND s.MCT_TYPE = '베이커리'
RETURN s.MCT_NM, u.WED_UE_CNT_RAT
ORDER BY u.WED_UE_CNT_RAT DESC
LIMIT 1



In [ ]:
print("""MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)\nWHERE c.name = '제주시' AND r.name = '삼도일동' AND s.MCT_TYPE = '베이커리'\nRETURN s.MCT_NM, u.WED_UE_CNT_RAT\nORDER BY u.WED_UE_CNT_RAT DESC\nLIMIT 1""")

MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시' AND r.name = '삼도일동' AND s.MCT_TYPE = '베이커리'
RETURN s.MCT_NM, u.WED_UE_CNT_RAT
ORDER BY u.WED_UE_CNT_RAT DESC
LIMIT 1
